In [106]:
import pandas as pd
import numpy as np
from datetime import datetime
from functools import reduce

import os
os.getcwd()

'/Users/reyashkadyan/Documents/GitHub/cp-decomposition'

In [2]:
# READING DATA
lookup_df = pd.read_excel('/Users/reyashkadyan/Desktop/Research/Stream-data/Lookuptable.xlsx')
q1_df = pd.read_excel('/Users/reyashkadyan/Desktop/Research/Stream-data/Streams BI Data_Query 1/West Bound/simple measures from BI_WB July 2020.xlsx', skiprows = 6, header = [0,1,2])
# q2_df = pd.read_excel('/Users/reyashkadyan/Desktop/Research/Stream-data/Streams BI Data_Query 2/Streams BI_NPI Raw Measures WB Jan 20-Jan 21.xlsx')


In [4]:
print("Shape of Q1", q1_df.shape)

# Drop all completely empty columns
q1_df.dropna(how='all', axis=1, inplace = True)

print("Revised hape of Q1", q1_df.shape)

Shape of Q1 (679, 279)
Revised hape of Q1 (679, 160)


In [6]:
def clean_header(col):
    col1, col2, col3 = col.split("|")
    
    if 'Travel Time' in col1:
        col1 = 'travel_time' 
    elif 'Posted Speed' in col1:
        col1 = 'post_speed'
    elif 'Travel Speed' in col1:
        col1 = 'travel_speed'
    elif col1 != '':
        col1 = '_'.join(col1.split(' ')[1:3]).lower()
    else:
        col1 = ''
        
    col2 = datetime.strftime(datetime.strptime(col2, "%d %b %y"), '%d-%m-%Y') if col2 != '' else ''
    col3 = col3.lower().replace(' ', '_') if col3.isupper() == False else ''
    # making new column label
    new_col = '%s%s%s' % ('%s ' % col1 if col1!= '' else '', '%s_' % col2 if col2!= '' else '','%s' % col3 if col3!= '' else '')
    return new_col.strip('_')

bound = 'W'

# FORMAT HEADER
column_text = ['%s%s%s' % ('%s|' % a if 'Unnamed' not in b else '|', '%s|' % b if 'Unnamed' not in b else '|', '%s' % c if 'Unnamed' not in c else '') for a, b, c in q1_df.columns]
print('Length of Columns Texts', len(column_text))

new_column_text = [clean_header(each) for each in column_text]
print('Length of New Columns Texts', len(new_column_text))

# setting new column names
q1_df.columns = new_column_text

# FORWARD FILL COMBINED CELLS
# Forward filling for selected columns
q1_df.row_labels = q1_df.row_labels.fillna(method = 'ffill')
q1_df.npi_avg_posted_speed = q1_df.npi_avg_posted_speed.fillna(method = 'ffill')
q1_df.npi_link_length = q1_df.npi_link_length.fillna(method = 'ffill')
q1_df.npi_link_type = q1_df.npi_link_type.fillna(method = 'ffill')

Length of Columns Texts 160
Length of New Columns Texts 160


In [8]:
# MELTING DATA
fixed_cols = ['row_labels', 'npi_avg_posted_speed', 'npi_link_length', 'npi_link_type', '15_minute_interval']

# extracting columns for each metric
avg_flow_cols = fixed_cols + [each for each in new_column_text if 'avg_flow' in each]
post_speed_cols = fixed_cols + [each for each in new_column_text if 'post_speed' in each]
travel_speed_cols = fixed_cols + [each for each in new_column_text if 'travel_speed' in each]
travel_time_cols = fixed_cols + [each for each in new_column_text if 'travel_time' in each]
congested_minutes_cols = fixed_cols + [each for each in new_column_text if 'congested_minutes' in each]

# melting all metric related columns into rows based on fixed columns
avg_flow_df = q1_df[avg_flow_cols].melt(fixed_cols, var_name='date', value_name='avg_flow')
post_speed_df = q1_df[post_speed_cols].melt(fixed_cols, var_name='date', value_name='post_speed')
travel_speed_df = q1_df[travel_speed_cols].melt(fixed_cols, var_name='date', value_name='travel_speed')
travel_time_df = q1_df[travel_time_cols].melt(fixed_cols, var_name='date', value_name='travel_time')
congested_minutes_df = q1_df[congested_minutes_cols].melt(fixed_cols, var_name='date', value_name='congested_minutes')

# fixing dates
avg_flow_df.date = avg_flow_df.date.apply(lambda x: x.split(' ')[1])
post_speed_df.date = post_speed_df.date.apply(lambda x: x.split(' ')[1])
travel_speed_df.date = travel_speed_df.date.apply(lambda x: x.split(' ')[1])
travel_time_df.date = travel_time_df.date.apply(lambda x: x.split(' ')[1])
congested_minutes_df.date = congested_minutes_df.date.apply(lambda x: x.split(' ')[1])

# check the shape of all metrics
# print(avg_flow_df.shape, post_speed_df.shape, travel_speed_df.shape, travel_time_df.shape, congested_minutes_df.shape)

# MERGING MELTED DATA
# pd.merge(avg_flow_df, post_speed_df, on = fixed_cols + ['date'])
data_frames = [avg_flow_df, post_speed_df, travel_speed_df, travel_time_df, congested_minutes_df]

# merging all the metric dataframes
df_merged = reduce(lambda  left,right: pd.merge(left, right, on = fixed_cols + ['date']), data_frames)

In [48]:
# MERGING WITH LOOKUP

# cleaning road names to be used in join with LookUp table
lookup_df.Road = lookup_df.Road.apply(lambda x: x.strip())
df_merged.row_labels = df_merged.row_labels.apply(lambda x: x.strip())
# lookup_df[lookup_df.Bound == 'W']

# merging with the LookUp
q1_df_merged = pd.merge(df_merged, lookup_df[lookup_df.Bound == bound], how = 'left', left_on = 'row_labels', right_on = "Road")

# CALCULATED COLUMNS
# splitting the time column into two
q1_df_merged[['StartTime', 'EndTime']] = q1_df_merged['15_minute_interval'].str.split(' - ', 1, expand=True)

# computing travel time for 10Kms
q1_df_merged['TT_10Kms'] = ((q1_df_merged.travel_time/q1_df_merged.npi_link_length)*1000)*10

# FINAL Q1 DATA
# filtering out desired columns
q1 = q1_df_merged[['date', 'O', 'D', 'npi_avg_posted_speed', 'npi_link_length', 'StartTime', 'EndTime', 'avg_flow', \
                   'travel_speed', 'travel_time', 'TT_10Kms', 'congested_minutes']]
# Rename columns as desired
q1.columns = ['Date', 'O', 'D', 'PostSpd', 'Length', 'StartTime', 'EndTime', 'Flow', 'Speed', 'AvgTT', 'TT_10Kms', 'CongMin']

# dropping the rows with empty Origin and Destination
q1 = q1.dropna(subset=['O', 'D'], axis = 0)

In [42]:
q1_df_merged[q1_df_merged.D.isna() == True].row_labels.unique()

array(['NPI(Metro) - 13 - Finucane Rd WB',
       'Shore St West WB between Grant St & Finucane Rd',
       'Finucane Rd WB between Willard Rd & Old Clevelan Old Cleve Ea Moreton Bay'],
      dtype=object)

In [108]:
def clean_header(col):
    col1, col2, col3 = col.split("|")
    
    if 'Travel Time' in col1:
        col1 = 'travel_time' 
    elif 'Posted Speed' in col1:
        col1 = 'post_speed'
    elif 'Travel Speed' in col1:
        col1 = 'travel_speed'
    elif col1 != '':
        col1 = '_'.join(col1.split(' ')[1:3]).lower()
    else:
        col1 = ''
        
    col2 = datetime.strftime(datetime.strptime(col2, "%d %b %y"), '%d-%m-%Y') if col2 != '' else ''
    col3 = col3.lower().replace(' ', '_') if col3.isupper() == False else ''
    # making new column label
    new_col = '%s%s%s' % ('%s ' % col1 if col1!= '' else '', '%s_' % col2 if col2!= '' else '','%s' % col3 if col3!= '' else '')
    return new_col.strip('_')

def prepare_Q1(file_name, bound, lookup_df):
    
    ## 1. READING DATA
    q1_df = pd.read_excel(file_name, skiprows = 6, header = [0,1,2])
    
    ## 2. DROP EMPTY COLUMNS
    q1_df.dropna(how='all', axis=1, inplace = True)
    
    ## 3. FORMAT HEADER
    column_text = ['%s%s%s' % ('%s|' % a if 'Unnamed' not in b else '|', '%s|' % b if 'Unnamed' not in b else '|', '%s' % c if 'Unnamed' not in c else '') for a, b, c in q1_df.columns]
    new_column_text = [clean_header(each) for each in column_text]

    # setting new column names
    q1_df.columns = new_column_text

    ## 4. FORWARD FILL COMBINED CELLS
    # Forward filling for selected columns
    q1_df.row_labels = q1_df.row_labels.fillna(method = 'ffill')
    q1_df.npi_avg_posted_speed = q1_df.npi_avg_posted_speed.fillna(method = 'ffill')
    q1_df.npi_link_length = q1_df.npi_link_length.fillna(method = 'ffill')
    q1_df.npi_link_type = q1_df.npi_link_type.fillna(method = 'ffill')
    
    ## 5. MELTING DATA
    fixed_cols = ['row_labels', 'npi_avg_posted_speed', 'npi_link_length', 'npi_link_type', '15_minute_interval']

    # extracting columns for each metric
    avg_flow_cols = fixed_cols + [each for each in new_column_text if 'avg_flow' in each]
    post_speed_cols = fixed_cols + [each for each in new_column_text if 'post_speed' in each]
    travel_speed_cols = fixed_cols + [each for each in new_column_text if 'travel_speed' in each]
    travel_time_cols = fixed_cols + [each for each in new_column_text if 'travel_time' in each]
    congested_minutes_cols = fixed_cols + [each for each in new_column_text if 'congested_minutes' in each]

    # melting all metric related columns into rows based on fixed columns
    avg_flow_df = q1_df[avg_flow_cols].melt(fixed_cols, var_name='date', value_name='avg_flow')
    post_speed_df = q1_df[post_speed_cols].melt(fixed_cols, var_name='date', value_name='post_speed')
    travel_speed_df = q1_df[travel_speed_cols].melt(fixed_cols, var_name='date', value_name='travel_speed')
    travel_time_df = q1_df[travel_time_cols].melt(fixed_cols, var_name='date', value_name='travel_time')
    congested_minutes_df = q1_df[congested_minutes_cols].melt(fixed_cols, var_name='date', value_name='congested_minutes')

    # fixing dates
    avg_flow_df.date = avg_flow_df.date.apply(lambda x: x.split(' ')[1])
    post_speed_df.date = post_speed_df.date.apply(lambda x: x.split(' ')[1])
    travel_speed_df.date = travel_speed_df.date.apply(lambda x: x.split(' ')[1])
    travel_time_df.date = travel_time_df.date.apply(lambda x: x.split(' ')[1])
    congested_minutes_df.date = congested_minutes_df.date.apply(lambda x: x.split(' ')[1])

    # check the shape of all metrics
    # print(avg_flow_df.shape, post_speed_df.shape, travel_speed_df.shape, travel_time_df.shape, congested_minutes_df.shape)

    ## 6. MERGING MELTED DATA
    data_frames = [avg_flow_df, post_speed_df, travel_speed_df, travel_time_df, congested_minutes_df]

    # merging all the metric dataframes
    df_merged = reduce(lambda  left,right: pd.merge(left, right, on = fixed_cols + ['date']), data_frames)
    
    ## 7. MERGING WITH LOOKUP
    # cleaning road names to be used in join with LookUp table
    lookup_df.Road = lookup_df.Road.apply(lambda x: x.strip())
    df_merged.row_labels = df_merged.row_labels.apply(lambda x: x.strip())

    # merging with the LookUp
    q1_df_merged = pd.merge(df_merged, lookup_df[lookup_df.Bound == bound], how = 'left', left_on = 'row_labels', right_on = "Road")

    ## 8. CALCULATED COLUMNS
    # splitting the time column into two
    q1_df_merged[['StartTime', 'EndTime']] = q1_df_merged['15_minute_interval'].str.split(' - ', 1, expand=True)

    # computing travel time for 10Kms
    q1_df_merged['TT_10Kms'] = ((q1_df_merged.travel_time/q1_df_merged.npi_link_length)*1000)*10

    ## 9. FINAL Q1 DATA
    # filtering out desired columns
    q1 = q1_df_merged[['date', 'O', 'D', 'npi_avg_posted_speed', 'npi_link_length', 'StartTime', 'EndTime', 'avg_flow', \
                       'travel_speed', 'travel_time', 'TT_10Kms', 'congested_minutes']]
    # Rename columns as desired
    q1.columns = ['Date', 'O', 'D', 'PostSpd', 'Length', 'StartTime', 'EndTime', 'Flow', 'Speed', 'AvgTT', 'TT_10Kms', 'CongMin']

    # dropping the rows with empty Origin and Destination
    q1 = q1.dropna(subset=['O', 'D'], axis = 0)
    
    ## 10. RETURN PRE-PROCESSED Q1 DATA
    return q1

In [107]:
# TESTED: Working for both West and East Bound

# load lookup table
Lookup = pd.read_excel('/Users/reyashkadyan/Desktop/Research/Stream-data/Lookuptable.xlsx')

# load the name of the file to be processed here
File_path = '/Users/reyashkadyan/Desktop/Research/Stream-data/Streams BI Data_Query 1/East Bound/simple measures from BI_EB Feb 2020.xlsx'

# change the traffic direction based on the file used above
Bound = 'E'

# Pre-process
Q1 = prepare_Q1(file_name = File_path, bound = Bound, lookup_df = Lookup)

# writing the file to excel
Q1.to_excel('type_your_file_name_here.xlsx', index = False)
